In [1]:
import numpy as np
import serial
import datetime as dt
import os
import time
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Process, Queue
import glob
import pandas as pd
import warnings

# imports from this module
from top_bottom_triggered.fast_animate import *
from top_bottom_triggered.commutator_utils import *

## Setup

In [2]:
port = 'COM4'

In [3]:
# generate a random order for mice to run, based on today's date 
# (will be the same even, eg, 1 hour later, as long as date is the same)

mice_to_run = ['gmou77', 'gmou78', 'gmou81', 'gmou83']

today = dt.datetime.now().date()
date_hash = int(dt.datetime(today.year, today.month, today.day).timestamp())
np.random.seed(date_hash)
np.random.shuffle(mice_to_run)
print(mice_to_run)

['gmou81', 'gmou78', 'gmou77', 'gmou83']


In [4]:
# inputs
subject = 'test'
date = dt.datetime.now().strftime('%Y%m%d')
time_in_minutes = 10 # go slightly longer than mkv to ensure complete overlap
base_path = R'D:\Jonah\trigger_testing'
# base_path = R'E:\Jonah\CeAMouse'
suffix = ''

path = os.path.join(base_path, f'{subject}\\{date}_{subject}')
# path = path.format(subject=subject, date=date)
n_samples = 4000  # how many thermistor samples to show
show_opto = False  # only set to true if there is a "stim" col in ino data

if not os.path.exists(path):
    os.makedirs(path)
    print(f'Created {path}')
else:
    print(f'Path {path} exists!')

Path D:\Jonah\trigger_testing\test\20230425_test exists!


In [5]:
# test serial port and check dac value
with serial.Serial(port, baudrate=115200, timeout=0.1) as ino:
    line = ino.readline().decode('utf-8').strip('\r\n')
    print(line)
    print(f'Data has {len(line.split(","))} elements')

12.6250,-20.8125,-100.1250,-0.06,-0.12,0.01,0,3.30
Data has 8 elements


In [65]:
# separates data collection cell from setup cells for safety

## Run the experiment!

TODO: 
* verify the 

In [ ]:
debug = False

# data collection cell, run this to start data storage!
q_downsample = 15  # leave at 15; how much to downsample rt output (doesnt affect saved data) (eg if 20, and ino at 1 khz, will be 50 hz)
fs = 500

# timing vars
start_time = dt.datetime.now()
one_mindelta = dt.timedelta(minutes=1)
exp_timedelta = time_in_minutes*one_mindelta # key var to be compared against (now - start_time)

# file vars
fname = f'{start_time.strftime("%Y%m%d")}_{subject}{suffix}.txt'
file_path = os.path.join(path, fname)

# flow vars
first_line = 1  # don't change
second_line = 0  # don't change
sync_sent = 0
header_max_attempts = 10  # 10 should be plenty. if fails, sth else is likely wrong.

# Open queue to animator
data_queue = Queue()
therm_data = np.zeros((n_samples,), dtype='float')
data_head_idx = 0  # to trace out data like an o-scope
q_downsample_counter = 0
animate_process = main_from_ipynb(data_queue, n_samples, int(fs/q_downsample))
    
# daq
try:
    with open(file_path, 'x') as file:
        with serial.Serial(port, baudrate=115200, timeout=0.1) as ino, serial.Serial('COM7', baudrate=9600, timeout=0.1) as sync_device:
            reader = ReadLine(ino)
            while (dt.datetime.now() - start_time) < exp_timedelta:  
                
                # Read the line
                line = reader.readline().decode('utf-8').strip('\r\n')
                
                # Remove the DEBUG output if present and debugging
                if debug:
                    line = line[:(line.find(',DEBUG:'))]

                # These checks get header and process it
                if first_line:
                    # Ask the arduino to print the header
                    ino.write('h'.encode('utf-8'))
                    
                    # Verify first line
                    first_line, second_line, header = first_line_check(header_max_attempts, file, ino)
                    
                    # Extract indices of values we're intersted in
                    header_len = len(header.split(','))
                    print(header)
                    thermistor_idx = [i for i,val in enumerate(header.split(',')) if (val=='therm' or val=='thermistor')][0]
                    if show_opto:
                        opto_idx = [i for i,val in enumerate(header.split(',')) if val==opto_header_name][0]
                    led_idxs = [i for i,val in enumerate(header.split(',')) if 'led' in val]
                    
                # Check the data lines have the correct amount of data
                # Returns false when good
                elif second_line:
                    second_line = second_line_check(line, header)
                
                # Assuming data looks good, start the sync device
                if not(first_line or second_line) and not(sync_sent):
                    print('sending start msg to sync device')
                    num = b"".join([packIntAsLong(int(time_in_minutes*60*30 + 300))])
                    sync_device.write(num)
                    print('msg sent')
                    sync_sent = 1
                    print(sync_device.readline().decode('utf-8'))
                    
                # Check for the typical (but rare) serial read issues
                if len(line) == 0:
                    print('Got empty line, continuing...')
                    continue
                elif len(line.split(',')) != header_len:
                    print('Got line with unexpected length (skipping):')
                    print(line)
                    continue
                else:  
                    # typical case -- write line directly to file
                    file.write(line)
                    file.write('\n')
                    
                
                # Prep thermistor data for animator
                therm_val = np.array(line.split(',')[thermistor_idx], dtype='float')
                therm_data[data_head_idx] = therm_val
                if show_opto:
                    opto_val = np.array(line.split(',')[opto_idx], dtype='float')

                # Show inhales and exhales in super janky way
#                 if np.array(line.split(',')[8], dtype='int') == 1:
#                     therm_data[data_head_idx] = 1000
#                 elif np.array(line.split(',')[9], dtype='int') == 1:
#                     therm_data[data_head_idx] = 0
                
                # Delete the oldest data to make it o-scope-like
                therm_data[(data_head_idx+100) % n_samples] = np.nan
                data_head_idx += 1
                data_head_idx = data_head_idx % n_samples

                # Increment downsample counter
                q_downsample_counter += 1
                q_downsample_counter = q_downsample_counter % q_downsample
                
                # Decide if sending to animator
                if q_downsample_counter == 0:
                    
                    # Extract syncs in real time
                    sync_tup = tuple([line.split(',')[idx] for idx in led_idxs])

                    if show_opto:
                        data_queue.put((therm_data, sync_tup, opto_val))
                    else:
                        data_queue.put((therm_data, sync_tup, 0))
                
            # After data collection finishes, close queue with empty tuple
            data_queue.put(tuple())

# Catch other unexpected errors            
finally:
    if animate_process.is_alive():
        data_queue.put(tuple())
    print('Done.')

time,led1,led2,led3,led4,yaw,roll,pitch,acc_x,acc_y,acc_z,therm,dac
sending start msg to sync device
msg sent
start sync pulses



## Post-experiment summaries

In [65]:
data = pd.read_csv(glob.glob(os.path.join(path, '*.txt'))[0])

In [66]:
therm_over_thresh_count = ((data.therm > 900) & (data.dac<=0.1)).sum()
therm_under_thresh_count = ((data.therm < 200) & (data.dac >= 3.25)).sum()
print(f'Therm over: {therm_over_thresh_count}')
print(f'Therm under: {therm_under_thresh_count}')
print(f'Time elapsed since start: {(dt.datetime.now() - start_time).seconds/60:0.1f} minutes')

Therm over: 4608
Therm under: 0
Time elapsed since start: 66.8 minutes
